In [ ]:
import rasterio
import numpy as np
import cv2
from osgeo import gdal
import tqdm
fp = r"E:\geoshapes\TOLUCA\zonas\todo.tif"
img = rasterio.open(fp)
array = img.read()
four_images=[array[2],array[1],array[0],array[3]]
stacked_images = np.stack(four_images, axis=-1)
H,W,D=stacked_images.shape
z=0
for i in tqdm.tqdm(range(int(H/1024))):
    for j in range(int(W/1024)):
        array1=stacked_images[1024*i:1024*(i+1),1024*j:1024*(j+1)]
        cv2.imwrite(r"E:\geoshapes\TOLUCA\imagenes1/t/zona_t_"+str(i)+"_"+str(j)+".png",array1)

In [ ]:

ds = gdal.Open(fp)
width = ds.RasterXSize
height = ds.RasterYSize
gt = ds.GetGeoTransform()
minx = gt[0]
miny = gt[3] + width*gt[4] + height*gt[5] 
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3] 